viernes 03 de mayo de 2019  
  
_Benjamín Hernández Cortés_ - _Juan Pablo Rojas Rojas_  
_Departamento de Ingeniería Informática (DIINF)_  
_Universidad de Santiago de Chile (USACH)_


## Laboratorio 2 - Fundamentos de Aprendizaje Profundo con Redes Neuronales
___

El presente código está orientado hacia la implementación de un perceptrón multicapa o red neuronal de múltiples capas (Multi-Layer Neural Network). En primera oportunidad, se utilizará el perceptrón para clasificar y emular compuertas lógicas AND, OR y XOR, tanto de 2 como de 4 entradas. Luego, se realizará una clasificación trabajando con el conjunto de datos [wine](https://archive.ics.uci.edu/ml/datasets/wine) provenientes de la [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) que contiene 13 atributos en total y que intentan describir clases de vinos provenientes de 3 cultivos distintos.


#### Importación de bibliotecas
---

Las bibliotecas a emplear son:
- **Numpy:** Herramienta de computación científica, que nos permitirá trabajar a través de vectores
- **Pandas:** Para la manipulación y lectura de datos
- **Matplotlib:** Para la visualización gráfica de diversos datos de interés
- **Itertools:** Como herramienta para iteración de objetos
- **Scikit-learn:** Para la obtención del dataset _wine_ y el uso de herramientas de evaluación de desempeño para los perceptrones multicapa.

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import itertools as its
import sklearn as skl

from sklearn.datasets import load_wine

#### Definición de funciones globales
---

Para el presente laboratorio se emplearan las siguientes funciones de forma global, las cual estan enfocadas al cálculo de ciertos valores a partir:
- **Numpy:** Herramienta de computación científica, que nos permitirá trabajar a través de vectores
- **Pandas:** Para la manipulación y lectura de datos
- **Matplotlib:** Para la visualización gráfica de diversos datos de interés
- **Itertools:** Como herramienta para iteración de objetos
- **Scikit-learn:** Para la obtención del dataset _wine_ y el uso de herramientas de evaluación de desempeño para los perceptrones multicapa.

In [ ]:
def tanh(x):
    return (1.0 - np.exp(-2 * x)) / (1.0 + np.exp(-2 * x))

def tanh_derivative(x):
    return (1 + tanh(x)) * (1 - tanh(x))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return 1 * (x > 0)

activation_functions = {
    'tanh': tanh,
    'sigmoid': sigmoid,
    'relu': relu
}

activation_functions_derivatives = {
    'tanh': tanh_derivative,
    'sigmoid': sigmoid_derivative,
    'relu': relu_derivative
}

#### Definición de la clase TruthTableGenerator
---

La clase TruthTableGenerator permite generar tablas de verdad para 3 tipos de compuertas lógicas: AND, OR y XOR.

En cuanto a las funciones definidas, se tiene:

| **Función**  | **Descripción**  |
| ------------ | ------------ |
| `generate_table(n_inputs, logic_gate)`  |  Genera una tabla de verdad de n-variables (*n_inputs*), basada en una compuerta lógica determinada (*logic_gate*)|

In [2]:
class TruthTableGenerator():
    
    def generate_table(n_inputs, logic_gate):
        table = its.product([0,1], repeat = n_inputs)
        table = pd.DataFrame(table)
        results = []
        
        for i in range(n_inputs ** 2):
            row = table.loc[[i]].values[0]
            if logic_gate == 'AND': results.append( int(all(row)) )
            elif logic_gate == 'OR': results.append( int(any(row)) )
            elif logic_gate == 'XOR':
                tmp_result = row[0] ^ row[1]
                for j in range(2, n_inputs):
                    tmp_result = tmp_result ^ row[j]
                results.append( tmp_result )
            else: return None
        
        table['result'] = results
        return table

#### Definición de la clase Layer
---
La clase Layer permite representar una de las capas que conforma al Perceptrón Multicapa. Requiere de 3 parámetros:

| <p style='text-align: left;'>**Parámetro**</p> | <p style='text-align: left;'>**Descripción**</p> |
| ------------ | ------------ |
| `number_of_inputs` |  <p style='text-align: justify;'>Valor númerico entero. Define la cantidad de entradas que recibe la capa, la cual debe ser mayor que 0. Por defecto, se define el número de entradas como 1.</p>|
| `number_of_neurons` | <p style='text-align: justify;'>Valor numérico entero. Define la cantidad de neuronas que posee la capa, la cual debe ser mayor que 0. Por defecto, se define el número de neuronas como 1.</p> |
| `activation_function` | <p style='text-align: justify;'>Cadena de caracteres. Define la función de activación que empleará la capa, la cual puede ser sigmoide (_sigmoid_) o ReLU (_relu_).</p> |

En cuanto a las funciones definidas, se tiene:

| <p style='text-align: left;'>**Función**</p>  | <p style='text-align: left;'>**Descripción**</p> |
| ------------ | ------------ |
| `calculate_output(input_vector)` |  <p style='text-align: justify;'>Realiza una predicción para un conjunto de datos (*input_vector*), empleando los valores actuales de los pesos (*weights*) asociados a la capa.</p>|


In [8]:
class MLP:
    def __init__(self, network_config, functions):
        self.layers = len(network_config)
        self.steps_per_epoch = 1
        self.network_config = network_config
        self.activity = []
        self.activity_derivative = []
        self.weights = []

        for layer in range(self.layers - 1):
            self.activity.append(activation_functions[functions[layer]])
            self.activity_derivative.append(activation_functions_derivatives[functions[layer]])
            w = np.random.rand(network_config[layer] + 1, network_config[layer + 1]) - 1
            w *= np.sqrt(1 / network_config[layer] + network_config[layer + 1])
            self.weights.append(w)
    
    def feed_forward(self, x):
        y = x

        for i in range(len(self.weights) - 1):
            activation = np.dot(y[i], self.weights[i])
            activity = self.activity[i](activation)
            activity = np.concatenate((np.ones(1), np.array(activity)))
            y.append(activity)

        activation = np.dot(y[-1], self.weights[-1])
        activity = self.activity[-1](activation)
        y.append(activity)
        return y
    
    def back_propagation(self, y, target, learning_rate):
        error = target - y[-1]
        delta_vector = [error * self.activity_derivative[-1](y[-1])]
        for i in range(self.layers-2, 0, -1):
            error = np.dot(delta_vector[-1], self.weights[i][1:].T)
            error = error * self.activity_derivative[i](y[i][1:])
            delta_vector.append(error)

        delta_vector.reverse()
        for i in range(len(self.weights)):
            layer = y[i].reshape(1, self.network_config[i] + 1)
            delta = delta_vector[i].reshape(1, self.network_config[i + 1])
            self.weights[i] += learning_rate * np.dot(layer.T, delta)
    
    def training(self, data, labels, learning_rate=0.1, epochs=100):
        ones = np.ones((1, data.shape[0]))
        Z = np.concatenate((ones.T, data), axis=1)
        
        for k in range(epochs):
            if (k+1) % 10000 == 0:
                print('epochs: {}'.format(k+1))
        
            sample = np.random.randint(X.shape[0])
            x = [Z[sample]]
            y = self.feed_forward(x)
            target = labels[sample]
            self.back_propagation(y, target, learning_rate)
    
    def predict_single_input(self, x):
        val = np.concatenate((np.ones(1).T, np.array(x)))
        for i in range(0, len(self.weights)):
            val = self.activity[i](np.dot(val, self.weights[i]))
            val = np.concatenate((np.ones(1).T, np.array(val)))
        return val[1]
    
    def predict(self, X):
        Y = np.array([]).reshape(0, self.arch[-1])
        for x in X:
            y = np.array([[self.predict_single_input(x)]])
            Y = np.vstack((Y,y))
        return Y
            
                
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

    

a = sklearn_to_df(load_wine())
b = np.array(TruthTableGenerator.generate_table(n_inputs=2, logic_gate='XOR'))

perceptron = MLP(data = b,
                 network_config = [2,2,1],
                 activation_functions_for_each_layer=['sigmoid', 'sigmoid'])

perceptron.training(10000)

data = np.array(b)
for i in range(data.shape[0]):
    input_vector = data[i,:-1]
    target_vector = data[i,-1]
    prediction = perceptron.predict(input_vector, target_vector)
    print("======\n input_vector -> {}\n target_vector -> {}\n prediction was -> {}\n =======".format(input_vector, target_vector, prediction))


ValueError: shapes (3,) and (1,1,1) not aligned: 3 (dim 0) != 1 (dim 1)